In [ ]:
import sys
import os
sys.path.append(os.path.join(sys.path[0], '../..'))

from pylamarck.algorithms.exhaustive_search import ExhaustiveSearchBasic,\
    ExhaustiveSearchExtended
from pylamarck.algorithms.random_sampling import RandomSampling
from pylamarck.production import RandomSequenceSearch, SequentialSearch,\
    ConstantSearch
from pylamarck.spaces.euclidean.production import RandomUniformSearch
from pylamarck.termination import MaxSteps, TimeLimit
from pylamarck.algorithms.hill_climbing import HillClimbing
from pylamarck.spaces.euclidean.production import RandomAdditiveMutation,\
    BoxConstraint, GaussianRn
from pylamarck.logging import EvaluationLogger, logme, convergence_demo2d,\
    goal_val_plot
from pylamarck.algorithms.random_walk import RandomWalk
import pylamarck.spaces.euclidean.test_functions_rn as test_rn

from numpy import argsort
from scipy.optimize import rosen

In [ ]:
search = ExhaustiveSearchBasic(range(10))
x = search.solve(lambda i: (i-5)**2)
print(x)

x = search.solve(lambda i: (i - 5) ** 2 + i)
print(x)

x = search.solve

In [ ]:
txt_set = ["aaa", "bb", "12", "Bee 1"]


In [ ]:
search = ExhaustiveSearchBasic(txt_set)

# Shortest string
x = search.solve(lambda i: len(i))
print(x)

# Smallest string in lexicographical order
x = search.solve(lambda i: i)
print(x)

# Smallest string in lexicographical order, more appropriate version
order = argsort(txt_set)
dictmap = dict(zip(txt_set, order))

search = ExhaustiveSearchExtended(txt_set, lambda i: dictmap[i])
x = search.solve(lambda i: i)
print(x)

# Largest string in lexicographical order
x = search.solve(lambda i: -i)
print(x)

# The simple version works because exhaustive search requires only a
# comparison operation


In [ ]:

# Random Sampling for Rosenbrock function
search = RandomSampling(RandomUniformSearch([-3, -3], [3, 3]),
                        TimeLimit(1))
x = search.solve(rosen)
print("Random sampling: ", x)

# Hill climbing for Rosenbrock function
cond = BoxConstraint([-3, -3], [3, 3]).get_checker()
ram = RandomAdditiveMutation(GaussianRn(2, 0.01),
                             cond=cond)
search = HillClimbing(ConstantSearch([-1.0, 1.0]),
                      ram, TimeLimit(1))
x = search.solve(rosen)
print("Hill climbing: ", x)

In [ ]:
# Optimization logging

el_rs_r2 = EvaluationLogger("Random sampling")
el_hc_r2 = EvaluationLogger("Hill climbing")

mincoord = [-3, -3]
maxcoord = [3, 3]

# Random Sampling for Rosenbrock function
search = RandomSampling(RandomUniformSearch(mincoord, maxcoord),
                        TimeLimit(1) | MaxSteps(5000))
x = search.solve(logme(el_rs_r2)(rosen))
print("Random sampling: ", x)

# optimization
cond = BoxConstraint(mincoord, maxcoord).get_checker()
ram = RandomAdditiveMutation(GaussianRn(2, 0.0, 0.1),
                             cond=cond)
search = HillClimbing(ConstantSearch([-1.0, 1.0]),
                      ram,
                      TimeLimit(1) | MaxSteps(5000))
x = search.solve(logme(el_hc_r2)(rosen))
print("Hill climbing: ", x)

convergence_demo2d([el_rs_r2, el_hc_r2], rosen, mincoord, maxcoord,
                   resample_evals_to=500)
goal_val_plot([el_rs_r2, el_hc_r2], mintoi=True, xtimestep='time', fmin=0)

# higher-dimensional Rosenbrock function
n_dim = 10

el_rs_rn = EvaluationLogger("Random sampling")
el_hc_rn = EvaluationLogger("Hill climbing")

mincoord = [-3 for _ in range(n_dim)]
maxcoord = [3 for _ in range(n_dim)]

search = RandomSampling(RandomUniformSearch(mincoord, maxcoord),
                        TimeLimit(1) | MaxSteps(50000))
x = search.solve(logme(el_rs_rn)(test_rn.frosenn))
print("Random sampling: ", x)

cond = BoxConstraint(mincoord, maxcoord).get_checker()
ram = RandomAdditiveMutation(GaussianRn(n_dim, 0.0, 0.1),
                             cond=cond)
search = HillClimbing(ConstantSearch([-1 for _ in range(n_dim)]),
                      ram,
                      TimeLimit(1) | MaxSteps(50000))

x = search.solve(logme(el_hc_rn)(test_rn.frosenn))
print("Hill climbing: ", x)

goal_val_plot([el_rs_rn, el_hc_rn], mintoi=True, xtimestep='time', fmin=0)


In [ ]:
# Random walk for comparison
n_dim = 2

el_rs_rn = EvaluationLogger("Random sampling")
el_hc_rn = EvaluationLogger("Hill climbing")
el_rw_rn = EvaluationLogger("Random walk")

mincoord = [-3 for _ in range(n_dim)]
maxcoord = [3 for _ in range(n_dim)]

search = RandomSampling(RandomUniformSearch(mincoord, maxcoord),
                        TimeLimit(1) | MaxSteps(500))
x = search.solve(logme(el_rs_rn)(test_rn.frosenn))
print("Random sampling: ", x)

cond = BoxConstraint(mincoord, maxcoord).get_checker()
ram = RandomAdditiveMutation(GaussianRn(n_dim, 0.0, 0.1),
                             cond=cond)

search = HillClimbing(ConstantSearch([-1 for _ in range(n_dim)]),
                      ram,
                      TimeLimit(1) | MaxSteps(500))

x = search.solve(logme(el_hc_rn)(test_rn.frosenn))
print("Hill climbing: ", x)

cond = BoxConstraint(mincoord, maxcoord).get_checker()
ram = RandomAdditiveMutation(GaussianRn(n_dim, 0.0, 0.1),
                             cond=cond)
search = RandomWalk(ConstantSearch([-1 for _ in range(n_dim)]),
                    ram,
                    TimeLimit(1) | MaxSteps(500))
x = search.solve(logme(el_rw_rn)(test_rn.frosenn))
print("Random walk: ", x)

if n_dim == 2:
    convergence_demo2d([el_rs_rn, el_hc_rn, el_rw_rn], rosen,
                       mincoord, maxcoord, resample_evals_to=500)
goal_val_plot([el_rs_rn, el_hc_rn, el_rw_rn],
              mintoi=True, xtimestep='time', fmin=0)


In [ ]:
# random sampling
search = RandomSampling(RandomSequenceSearch(txt_set), MaxSteps(20))

# Longest string
x = search.solve(lambda i: len(i))
print(x)

# Smallest in lexicographical order
order = argsort(txt_set)
dictmap = dict(zip(txt_set, order))
search = RandomSampling(RandomSequenceSearch(txt_set), MaxSteps(20),
                        lambda i: dictmap[i])
x = search.solve(lambda i: i)
print(x)

# Exhaustive search via Random Sampling
search = RandomSampling(SequentialSearch(txt_set), MaxSteps(len(txt_set)))
x = search.solve(lambda i: len(i))
print(x)

# test for a small interval
search = RandomSampling(RandomSequenceSearch(range(10)), MaxSteps(20))
x = search.solve(lambda i: (i - 5) ** 2)
print("Small interval: ", x)

# test for a wide interval
search = RandomSampling(RandomSequenceSearch(range(200000000)), MaxSteps(20))
x = search.solve(lambda i: (i - 500) ** 2)
print("Large interval: ", x)

# time limit example
search = RandomSampling(RandomSequenceSearch(range(200000000)), TimeLimit(1))
x = search.solve(lambda i: (i - 500) ** 2)
print("Wide interval (time limit): ", x)

# exhaustive search again
search = RandomSampling(SequentialSearch(range(10)), MaxSteps(10))
x = search.solve(lambda i: (i - 5) ** 2)
print("Small interval (sequentially): ", x)